In [38]:
import numpy as np
import pandas as pd

# Artificial Neurons: 
Uses a decision function, sigma of (z), to determine whether the input z belongs in class 0 or 1.

z is actually defined as a dot product between a weights vector, W, and a inputs vector, X. 

If z is bigger than some threshold value T, then z is classified as class 0. Otherwise, z is classified into class 1.

In [8]:
# Simple Implementation

class ArtificialNeuron:
  """
  All matrices/vectors that interact with this class must be a numpy array.
  
  Implementation of an artificial neuron
  """
  def __init__(self, W, T):
    self.weights = W
    self.threshold = T
  
  def __init__(self, num, T):
    self.weights = np.random.rand(num)
    self.threshold = T
  
  def classify(self, inputs):
    """
    Returns if the dot product between the weights and the inputs is greater than the threshold value.
    """
    assert inputs.shape == self.weights.shape, "Inputs does not match weights shape"
    z = np.dot(self.weights, inputs)
    if z >= self.threshold:
      return 1
    else:
      return 0

# Testing our code out
neuron = ArtificialNeuron(np.array(2), 0.8)
print("neuron output on (-2,2):", neuron.classify(np.array([-2,2])))
    
  


neuron output on (-2,2): 1


# Development of the Perceptron
Note that, if we were to take the threshold value and subtract it from the dot product between the weights and the input vector, we could then simply see if the difference is greater than 0.

This idea leads to the concept of biases, and ultimately lands us in Perceptron land. Take a look at my implementation of the Perceptron, heavily derived from the Artificial Neuron class.

## train

In [14]:
# Simple Implementation

class Perceptron:
  """
  All matrices/vectors that interact with this class must be a numpy array.
  
  Implementation of the Perceptron Algorithm
  """
  
  def __init__(self, num, alpha):
    """Randomly initializes weights and sets bias to 0. You must specify the 
    size of the weights vector (which should match the size of each training 
    sample). You also must choose the learning rate"""
    self.weights = np.random.rand(num)
    self.bias = 0
    self.learning_rate = alpha
  
  def classify(self, inputs):
    """
    Returns if the dot product between the weights and the inputs is greater than the threshold value.
    """
    assert inputs.shape == self.weights.shape, "Inputs does not match weights shape"
    z = np.dot(self.weights, inputs)
    if z + self.b >= 0:
      return 1
    else:
      return 0
  
  def train(self, training_set, epochs):
    """
    This method expects n individual training data samples, and uses them
    to perform weight and bias updates. 

    The shape of each individual sample in the training dataset should match 
    the shape of our weights vector.

    The training_set should be a list of tuples, where index 0 of each tuple
    is an input value, and index 1 is the correct label. 
    """
    
    # for-loop implementation
    for _ in epochs:
      for individual_sample in training_set:
        input_value, true_label = individual_sample
        perceptron_output = np.dot(self.weights, input_value) + self.bias
        predicted_label = None
        if perceptron_output >= 0:
          predicted_label = 1
        else:
          predicted_label = 0
        
        learning_rate_times_label_delta = self.learning_rate * (true_label - predicted_label)
        weight_update = learning_rate_times_label_delta * input_value
        bias_update = learning_rate_times_label_delta

        self.weights = self.weights + weight_update
        self.bias = self.bias + bias_update
    
  def get_weights_and_bias(self):
    """Returns weights and bias as a tuple"""
    return (self.weights, self.bias)
    
  
  

In [37]:
# Testing our perceptron algorithm implementation
perceptron = Perceptron(2,0.01)
print(perceptron.get_weights_and_bias())

(array([0.24440122, 0.49590235]), 0)
